# Hot fix for function in civiActivityReport_selectBestTrans

In [1]:
import os
import pandas as pd
import numpy as np
import re
import datetime
import itertools
from datetimerange import DateTimeRange

In [ ]:
#original function; works some of the time, but there are cases where it doesn't
def intake_file(activityReport):

    #I removed some data munging here, as this should be done in another module or the calling ipynb; munging removed denoted w/ "*"

    #activityReport.columns = [i.replace(' ','_')+'_act' for i in list(activityReport.columns)]*

    #trim all string fields
    #strip all whitespace from each cell
    #activityReport = activityReport.map(lambda x: x.strip() if isinstance(x,str) else x) *

    #NOTE: Activity_Date_act field DOES NOT provide seconds
    #activityReport = activityReport.assign(Activity_Date_DT_act = pd.to_datetime(activityReport['Activity_Date_act'], format = '%Y-%m-%d %H:%M')) *

    try:
        activityReport = activityReport.drop_duplicates(ignore_index = True, subset = ['Activity_Type_act', 'Subject_act', 'Activity_Date_act', 'Activity_Status_act', 'Activity_Date_DT_act','Target_Email_act'])

        # ### SELECT BEST TRANSACTION LOGIC
        #NOTE: real dupes have multiple entries on fields: 'Target_Email_act','Activity_Date_DT_act','Activity_Type_act'
        #NOTE 'Activity_Type_act' = specifies if change is for status or type
        # looks like the index is preserved on a groupby
        df_grouped = activityReport.groupby(['Target_Email_act','Activity_Date_DT_act','Activity_Type_act']).filter(lambda x: len(x) > 1)

    except ValueError as e:
        raise ValueError(f'fields in this version of the dataframe do not coincide with the code in civiActivityReport_selectBestTrans.py: {e}')


    #assign a row value to ea group member: used later for selection
    df_grouped['count'] = df_grouped.groupby(['Target_Email_act','Activity_Date_DT_act','Activity_Type_act']).cumcount()+1


    # Issues to handle:
    # - multiple records for different versions of Target_Name_act (yet Target_Email_act is the same) <- will need to check that this doesn't delete Family account members
    # - records made by different Source_Email_act ie systematic records made by CIVI

    df_grouped['from'] = df_grouped['Subject_act'].str.extract(r'from\s(\w+)') #str.extract(r'from\s(\w+)')
    df_grouped['to'] = df_grouped['Subject_act'].str.extract(r'to\s(\w+)') #str.extract(r'to\s(\w+)')


    # The record to keep is that where the "From" = "To" of the companion line of the group. There are only at most two entries with the same Start_dt, so I only need to worry about passing back the derived "To" and "From" fields twice. Either of row = 1 or row = 2 can be the best one to keep. Essentially I'm searching for the best determinite record of the status going forward, ignoring the journey (implying that accurate status is more important than accurate/comprehensive journey).
    # 
    # Will need to offset forwards and backwards. Ea row will then have a pair of offset values (from the row above and the one beneath). Depending on the "count" value (ie 1,2), only one from the pair will be relevant and tested to determine the "best" and "final" value.

    df_grouped.sort_values(['Target_Email_act','Activity_Type_act','Activity_Date_DT_act'],inplace= True)
    # -1 = the row below (lead); +1 = the row above (lagged)
    df_grouped[['from_-1','from_1']] = df_grouped['from'].shift(periods = [-1,1])
    df_grouped[['to_-1','to_1']] = df_grouped['to'].shift(periods = [-1,1])

    #do the same for Subject_act: this will help me sift through cases where Trial expirations are conflicting with new trial or membership starts (some kind of rollover - ex. taylor.m.posey@outlook.com)
    df_grouped[['Subject_act_-1','Subject_act_1']] = df_grouped['Subject_act'].shift(periods = [-1,1])


    # Handle cases where the -1 or +1 shift are irrelevant: have to do with different email addresses, as detectable by the 'count' field (ie 1,2)
    # This will actually be handled by a filter

    df_grouped.reset_index(names = 'index', inplace= True)
    return df_grouped

In [2]:
os.chdir('/home/mofongo/Documents/ghfc/membershipReportsCIVI/membershipReportingLogicSampleReports')

activityReport2 = pd.read_csv('./selectActivityReport_02092025.csv')

activityReport2.columns = [i.replace(' ','_')+'_act' for i in list(activityReport2.columns)]

activityReport2 = activityReport2.assign(Activity_Date_DT_act = pd.to_datetime(activityReport2['Activity_Date_act'], format = '%Y-%m-%d %H:%M'))

In [3]:
activityReport = activityReport2.drop_duplicates(ignore_index = True, subset = ['Activity_Type_act', 'Subject_act', 'Activity_Date_act', 'Activity_Status_act', 'Activity_Date_DT_act','Target_Email_act'])

In [19]:
activityReport.shape

(546, 10)

In [7]:
# ### SELECT BEST TRANSACTION LOGIC
#NOTE: real dupes have multiple entries on fields: 'Target_Email_act','Activity_Date_DT_act','Activity_Type_act'
#NOTE 'Activity_Type_act' = specifies if change is for status or type
# looks like the index is preserved on a groupby
df_grouped = activityReport.groupby(['Target_Email_act','Activity_Date_DT_act','Activity_Type_act']).filter(lambda x: len(x) > 1)

#    except ValueError as e:
#        raise ValueError(f'fields in this version of the dataframe do not coincide with the code in civiActivityReport_selectBestTrans.py: {e}')

In [8]:
#assign a row value to ea group member: used later for selection
df_grouped['count'] = df_grouped.groupby(['Target_Email_act','Activity_Date_DT_act','Activity_Type_act']).cumcount()+1


# Issues to handle:
# - multiple records for different versions of Target_Name_act (yet Target_Email_act is the same) <- will need to check that this doesn't delete Family account members
# - records made by different Source_Email_act ie systematic records made by CIVI

df_grouped['from'] = df_grouped['Subject_act'].str.extract(r'from\s(\w+)') #str.extract(r'from\s(\w+)')
df_grouped['to'] = df_grouped['Subject_act'].str.extract(r'to\s(\w+)') #str.extract(r'to\s(\w+)')

In [9]:
df_grouped.sort_values(['Target_Email_act','Activity_Type_act','Activity_Date_DT_act'],inplace= True)

In [20]:
len(df_grouped)

0

In [11]:
df_grouped.columns

Index(['Assignee_Name_act', 'Target_Name_act', 'Source_Email_act',
       'Target_Email_act', 'Activity_Type_act', 'Subject_act',
       'Activity_Date_act', 'Activity_Status_act', 'Activity_Details_act',
       'Activity_Date_DT_act', 'count', 'from', 'to'],
      dtype='object')

In [15]:
len(df_grouped['from'])

0

In [ ]:
df_grouped['from'].shift(periods = [-1,1])

In [12]:
    # -1 = the row below (lead); +1 = the row above (lagged)
df_grouped[['from_-1','from_1']] = df_grouped['from'].shift(periods = [-1,1])


ValueError: Columns must be same length as key

In [ ]:

df_grouped[['to_-1','to_1']] = df_grouped['to'].shift(periods = [-1,1])